In [ ]:
import numpy as np
import cv2
import tensorflow as tf

This function perform multiple augmentations on the input image like horizontal flip, rotation, brightness and contrast changes, and zoom. It is not a complete solution and you can always add more augmentations to the function as per your requirement.

In [ ]:
def augment_retinal_image(image):
    # Randomly flipping the image horizontally
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        
    # Randomly rotating the image by a small angle
    angle = np.random.randint(-5, 5)
    M = cv2.getRotationMatrix2D((image.shape[1]/2, image.shape[0]/2), angle, 1)
    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    
    # Randomly applying brightness and contrast changes
    alpha = np.random.uniform(0.5, 1.5)
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=np.random.randint(-50, 50))

    # Randomly zooming the image
    zoom = np.random.uniform(0.9, 1.1)
    M = cv2.getRotationMatrix2D((image.shape[1]/2, image.shape[0]/2), 0, zoom)
    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    return image

In [ ]:
def create_dataset(parent_dir, augment_fn=None):
    # Create a list of all subdirectories within the parent directory
    subdirs = [d for d in tf.io.gfile.listdir(parent_dir) if tf.io.gfile.isdir(f"{parent_dir}/{d}")]
    
    # Create a dataset of all image paths within the subdirectories
    image_paths = []
    for subdir in subdirs:
        image_paths += tf.io.gfile.glob(f"{parent_dir}/{subdir}/*.jpg")
    images_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    
    # Read and decode the images
    def read_and_decode(path):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        if augment_fn:
            image = augment_fn(image)
        return image
    images_ds = images_ds.map(read_and_decode, num_parallel_calls=tf.data.AUTOTUNE)
    
    return images_ds

This function takes two arguments, the path of the parent directory and an optional function for data augmentation. It first creates a list of all subdirectories within the parent directory, it then creates a dataset of all image paths within the subdirectories. It then uses the tf.data.Dataset.map() method to read and decode the images, applying the optional data augmentation function, if provided, before returning the dataset.

In [ ]:
augment_fn = lambda x: augment_retinal_image(x)
images_ds = create_dataset("path/to/parent/dir", augment_fn)